# Анализатор тональности отзывов с объяснением.

## Импорты

In [1]:
!pip install -q "langchain>=0.3.0" langchain-mistralai langchain-community chromadb kagglehub tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 11.8 MB/s eta 0:

In [2]:
!pip install -U openai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.9/471.9 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.6 which is incompatible.


In [37]:
import os
import json
import re
import time
import nltk

import kagglehub
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

from google.colab import userdata
from google.colab import files

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings

from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

from google.api_core import exceptions

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error,
    precision_score, recall_score, f1_score,
    classification_report, cohen_kappa_score
)
from sklearn.preprocessing import LabelEncoder
from scipy.stats import spearmanr, pearsonr
import re
import time
from tqdm import tqdm
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

 ## Загрузка данных

In [4]:
path = kagglehub.dataset_download("fahadrehman07/movie-reviews-and-emotion-dataset")
print("Path to dataset files:", path)

csv_path = os.path.join(path, 'Movies_Reviews_modified_version1.csv')
df = pd.read_csv(csv_path)
df.head()

100%|██████████| 31.4M/31.4M [00:00<00:00, 114MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/fahadrehman07/movie-reviews-and-emotion-dataset/versions/3


,Unnamed: 0,Ratings,Reviews,movie_name,Resenhas,genres,Description,emotion
0,0,3.0,"It had some laughs, but overall the motivation...",Waiting to Exhale,"Riu algumas risadas, mas no geral a motivação ...","['Comedy', 'Drama', 'Romance']","Based on Terry McMillan's novel, this film fol...",anticipation
1,1,4.0,"WAITING TO EXHALE Waiting, and waiting, and wa...",Waiting to Exhale,"ESPERANDO PARA EXALAR Esperando, e esperando, ...","['Comedy', 'Drama', 'Romance']","Based on Terry McMillan's novel, this film fol...",anticipation
2,2,4.0,"Angela Basset was good as expected, but Whitne...",Waiting to Exhale,"Angela Basset foi boa como o esperado, mas Whi...","['Comedy', 'Drama', 'Romance']","Based on Terry McMillan's novel, this film fol...",anticipation
3,3,5.0,"The movie is okay, mediocre might even be the ...",Waiting to Exhale,"O filme é bom, medíocre pode até ser a palavra...","['Comedy', 'Drama', 'Romance']","Based on Terry McMillan's novel, this film fol...",anticipation
4,4,5.0,I got an opportunity to see Waiting To Exhale ...,Waiting to Exhale,Tive a oportunidade de ver Waiting To Exhale p...,"['Comedy', 'Drama', 'Romance']","Based on Terry McMillan's novel, this film fol...",anticipation


In [5]:
# берём 500 для экспериментов
sample_df = df.sample(n=10000, random_state=42)

# делим на train / test
train_df, test_df = train_test_split(sample_df, test_size=0.2, random_state=42)

print("Train size:", len(train_df), "Test size:", len(test_df))
print("Total size:", len(sample_df))

Train size: 8000 Test size: 2000
Total size: 10000


## Инициализация моделей

In [38]:
# # ключ ChatGPT (OpenAI)
# openai_api_key = userdata.get("OPENAI_API_KEY")
# os.environ["OPENAI_API_KEY"] = openai_api_key
# print("OpenAI key present:", bool(openai_api_key))

api_key = userdata.get("MISTRAL_API_KEY")
print("API key present:", bool(api_key))

API key present: True


In [39]:
llm = ChatMistralAI(
    model="open-mistral-7b",
    api_key=api_key,
    temperature=0.2,
)

light_llm = ChatMistralAI(
    model="mistral-tiny",
    api_key=api_key
)

emb = MistralAIEmbeddings(
    model="mistral-embed",
    api_key=api_key,
)

tokenizer.json: 0.00B [00:00, ?B/s]

In [7]:
# # Основной LLM для RAG
# llm = ChatOpenAI(
#     model="gpt-4.1-mini",
#     temperature=0.2,
# )

# # Лёгкий LLM для фильтрации / классификаций
# light_llm = ChatOpenAI(
#     model="gpt-4.1-mini",
#     temperature=0.0,
# )

# emb = OpenAIEmbeddings(
#     model="text-embedding-3-small",
#     openai_api_key=openai_api_key,
# )

## Транспорт датасета в документ

Для обучающей выборки.

In [8]:
docs_train = []

for _, row in train_df.iterrows():
    genres = row["genres"]
    if isinstance(genres, (list, tuple)):
        genres_str = ", ".join(genres)
    else:
        genres_str = str(genres)

    text = f"""Movie: {row['movie_name']}
Rating: {row['Ratings']}
Review: {row['Reviews']}
Genres: {genres_str}
Emotion: {row['emotion']}
Description: {row['Description']}
""".strip()

    doc = Document(
        page_content=text,
        metadata={
            "movie_name": row["movie_name"],
            "rating": float(row["Ratings"]),
            "emotion": row["emotion"],
        },
    )
    docs_train.append(doc)

print("Всего документов:", len(docs_train))
print("Пример документа:\n", docs_train[0].page_content)


Всего документов: 8000
Пример документа:
 Movie: Swing Shift
Rating: 4.0
Review: Jonathan Demme directed this period piece set during World War II, where women were recruited to take over from men in airplane making factories, because the men had to serve(unless they were declared 4-F).Goldie Hawn, Christine Lahti, and Holly Hunter play the women, while Ed Harris, Fred Ward, and Kurt Russell play the men. The women must overcome the sexism and skepticism from management, and some of the men left behind. With their husbands gone, the women find that their increased responsibility makes them more involved with the world, but also gives way to some temptations as well...Surprisingly bland, even lifeless film feels longer than it is, though it does have a good cast, story doesn't hold viewer interest much, and it feels like a wasted opportunity to portray an important part of the home front aspect of the war.
Genres: ['Comedy', 'Drama', 'Romance']
Emotion: optimism
Description: A woman fin

И для тестовой выборки.

In [9]:
docs_test = []

for _, row in test_df.iterrows():
    genres = row["genres"]
    if isinstance(genres, (list, tuple)):
        genres_str = ", ".join(genres)
    else:
        genres_str = str(genres)

    text = f"""Movie: {row['movie_name']}
Rating: {row['Ratings']}
Review: {row['Reviews']}
Genres: {genres_str}
Emotion: {row['emotion']}
Description: {row['Description']}
""".strip()

    doc = Document(
        page_content=text,
        metadata={
            "movie_name": row["movie_name"],
            "rating": float(row["Ratings"]),
            "emotion": row["emotion"],
        },
    )
    docs_test.append(doc)

print("Всего документов:", len(docs_test))
print("Пример документа:\n", docs_test[0].page_content)

Всего документов: 2000
Пример документа:
 Movie: +1
Rating: 5.0
Review: 'Shadow Walkers' (or '+1' as it's also known) is an odd little number at the best of times. It never really seems to know what it is. I guess the one thing everyone can agree on is that it's a 'B-movie.' Therefore, most likely everyone concerned with making it knew that it was never going to be a massively commercial box office smash. At best it might achieve 'cult status' on DVD.Therefore, it doesn't have a budget and no 'known' actors to speak of. It's about a meteor that crashes into a small town in America, creating weird phenomenon amidst a local college party. Soon the guests find things are going a little crazy and they might not make it to morning alive. It starts off like some sort of adult-humour piece like American Pie, then it progresses into a looping version of Groundhog Day. There's also a fair bit of 'gratuitous' nudity which (judging from what I've read online) seems to have put a lot of people off

## Создаем векторную базу данных

In [41]:
# vectorstore.delete_collection()

def create_batches(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

Инициализация параметров.

In [42]:
BATCH_SIZE = 50
DELAY_SECONDS = 1

doc_batches = list(create_batches(docs_train, BATCH_SIZE))
total_batches = len(doc_batches)

print(f"Общее количество документов: {len(docs_train)}")
print(f"Разбито на {total_batches} пакетов по {BATCH_SIZE} документов.")

Общее количество документов: 8000
Разбито на 160 пакетов по 50 документов.


Добавление пакетов.

In [43]:
# создаем базу данных Chroma, используя первый пакет.
vectorstore = Chroma.from_documents(
    documents=doc_batches[0],
    embedding=emb,
    collection_name="movie_reviews_rag",
    persist_directory="./chroma_db"
)

# добавляем оставшиеся пакеты
if total_batches > 1:
    for i, batch in enumerate(doc_batches[1:]):
        current_batch_num = i + 2
        time.sleep(DELAY_SECONDS)
        try:
            vectorstore.add_documents(documents=batch)
        except exceptions.ResourceExhausted as e:
            print(f"Ошибка: {e}")
            break

Создаем ретривер.

In [44]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

Смотрим, сколько документов.

In [45]:
num_docs = vectorstore._collection.count()
print("Всего документов в коллекции:", num_docs)

Всего документов в коллекции: 8000


## Промптим

Промптим модель.

In [46]:
prompt = ChatPromptTemplate.from_template("""
You are a movie review rating assistant.

You are given:
- CONTEXT: past reviews with their ratings.
- A NEW REVIEW written by a user.

Use the CONTEXT and the rating scale below to infer an appropriate rating from 1 to 10
for the NEW REVIEW and explain your reasoning.

Rating scale for movie reviews (1–10):

1 — Awful: almost no redeeming qualities, extremely negative experience.
2 — Very bad: many serious problems, very few minor positives.
3 — Bad: mostly negative, some aspects may be tolerable but overall disappointing.
4 — Below average: noticeable flaws, more negatives than positives, would not recommend.
5 — Mixed/average: clear pros and cons, watchable but not impressive.
6 — Decent: slightly above average, more positives than negatives, some enjoyable elements.
7 — Good: solid overall impression, flaws are minor and do not ruin the experience.
8 — Very good: strong positive emotions, well-made, would recommend to most people.
9 — Excellent: highly engaging, memorable, only very minor issues.
10 — Outstanding/masterpiece: exceptional in almost every aspect, would strongly recommend and rewatch.

CONTEXT:
{context}

NEW REVIEW:
{question}

Return:
1) "Rating: X" where X is an integer from 1 to 10.
2) "Explanation: ..." — short explanation based on similarities with CONTEXT and the rating scale.

Answer strictly in English.
""".strip())

rag_chain = (
    RunnableParallel(
        context=retriever,
        question=RunnablePassthrough()
    )
    | prompt
    | light_llm
)

## Результат работы

Зададим пример не из датасета и посмотрим на ответ модели.

In [47]:
new_review = "The movie was visually beautiful, but the story was slow and I got bored by the end."
resp = rag_chain.invoke(new_review)
print(resp.content)

Rating: 6
Explanation: The review mentions that the movie was visually beautiful but the story was slow and boring. This suggests that there are some positive aspects, such as the visuals, but the pacing and overall engagement were not strong enough to warrant a higher rating. This is similar to the "Love" movie in the context, where the visuals and idea were praised but the pacing and story were criticized. However, unlike "Love," the new review does not mention any major flaws or deep issues with the movie, making it a slightly above-average film (6 on the scale).


Ответ и объяснение выглядят довольно правдоподобно. Теперь посмотрим, какие документы модель извлекла из RAG.

In [48]:
retrieved = retriever.invoke(new_review)
for i, d in enumerate(retrieved, 1):
    print(f"\n--- Doc {i} ---")
    print(d.page_content[:400], "...")


--- Doc 1 ---
Movie: Samsara
Rating: 3.0
Review: I did watch the whole movie. The filmmakers tried to create a spiritual documentary without any understanding of foreign cultures and traditions. The result is a pathetic and very shallow bunch of scenes. It's like posting random images with 'deep' quotes on Facebook. The movie is heavily targeted on average uneducated narrow-minded people. That's too bad many pe ...

--- Doc 2 ---
Movie: Magnolia
Rating: 3.0
Review: This is no BOOGIE NIGHTS. This is what happens when one of these young hot-shot writer-directors get too much power too soon. A three hour yawn-fest, this film drones on and on with nowhere to go and nothing relevant to say about anything.  It's an exercise in super-arsty filmmaking with no reason to watch other than to observe the director's almost ridiculous l ...

--- Doc 3 ---
Movie: The Fall
Rating: 4.0
Review: I have to say I had huge hopes for The Fall when I saw it last night, and I was sorely disappointed. If you g

## Метрики

Оценим качество выставления оценки отзыву по актуальным метрикам.

In [51]:
def extract_predicted_rating(text: str) -> int | None:
    """
    Пытается вытащить целое число 1-10 из строки, включая числа с плавающей точкой.
    """

    m = re.search(r"Rating\s*:\s*(\d+(?:\.\d+)?)", text, re.IGNORECASE)
    if m:
        rating = float(m.group(1))
        return int(rating)

    m = re.search(r"\b([1-9]|10)\b", text)
    if m:
        return int(m.group(1))

    return None


def evaluate_rag_model_batch(
    test_df,
    rag_chain,
    max_samples: int | None = 500,
    batch_size: int = 50,
    max_concurrency: int = 5,
    base_sleep_sec: float = 0.0,
):
    """
    Оценивает модель, используя пакетные вызовы (.batch) с прогрессбаром
    и обработкой rate limit (429).
    """

    eval_df = test_df.sample(n=max_samples, random_state=42)

    reviews_list = eval_df["Reviews"].tolist()
    y_true_list = eval_df["Ratings"].astype(int).tolist()
    n = len(reviews_list)

    raw_outputs: list[str | None] = [None] * n

    # батчевый вызов модели
    for start in tqdm(range(0, n, batch_size), desc="Оценка RAG по батчам"):
        end = min(start + batch_size, n)
        batch_inputs = reviews_list[start:end]

        while True:
            try:
                batch_responses = rag_chain.batch(
                    batch_inputs,
                    config={"max_concurrency": max_concurrency},
                )
                break

            except Exception as e:
                msg = str(e)
                # повторяем этот же батч
                if "rate_limit_exceeded" in msg or "Rate limit reached" in msg:
                    m = re.search(r"Please try again in (\d+(\.\d+)?)s", msg)
                    delay = 60
                    print(
                        f"\n Rate limit на батче [{start}:{end}]. "
                        f"Ждём {delay:.2f} сек и повторяем..."
                    )
                    time.sleep(delay)
                    continue

                print(f"\n Ошибка на батче [{start}:{end}]: {e}")
                break

        # сохраняем результаты
        if "batch_responses" in locals():
            for i, resp in enumerate(batch_responses):
                text = resp.content if hasattr(resp, "content") else str(resp)
                raw_outputs[start + i] = text
            del batch_responses

        # пауза между батчами
        if base_sleep_sec > 0:
            time.sleep(base_sleep_sec)

    # парсинг
    y_pred: list[int | None] = []
    for idx, text in enumerate(tqdm(raw_outputs, desc="Парсинг ответов")):
        if text is None:
            y_pred.append(None)
            continue

        pred_rating = extract_predicted_rating(text)
        if pred_rating is None:
            print(
                f" Could not parse rating for example #{idx+1}. "
                f"Raw output: '{text[:100]}...'"
            )
        y_pred.append(pred_rating)

    # метрики
    valid_indices = [i for i, pred in enumerate(y_pred) if pred is not None]
    y_true = np.array(y_true_list)[valid_indices]
    y_pred_final = np.array([y_pred[i] for i in valid_indices])

    metrics = {
            'mae': float(mean_absolute_error(y_true, y_pred)),
            'mse': float(mean_squared_error(y_true, y_pred)),
            'rmse': float(np.sqrt(mean_squared_error(y_true, y_pred))),
            'accuracy_exact': float(np.mean(y_true == y_pred)),
            'accuracy_tol1': float(np.mean(np.abs(y_true - y_pred) <= 1)),
            'accuracy_tol2': float(np.mean(np.abs(y_true - y_pred) <= 2)),
            'pearson_corr': float(pearsonr(y_true, y_pred)[0]),
            'spearman_corr': float(spearmanr(y_true, y_pred)[0]),
            'cohen_kappa': float(cohen_kappa_score(y_true, y_pred)),
            'f1_macro': float(f1_score(y_true, y_pred, average='macro', zero_division=0)),
            'f1_micro': float(f1_score(y_true, y_pred, average='micro', zero_division=0)),
            'f1_weighted': float(f1_score(y_true, y_pred, average='weighted', zero_division=0)),
            'precision_macro': float(precision_score(y_true, y_pred, average='macro', zero_division=0)),
            'recall_macro': float(recall_score(y_true, y_pred, average='macro', zero_division=0))
    }

    try:

        min_rating = min(min(y_true), min(y_pred))
        max_rating = max(max(y_true), max(y_pred))
        weights = np.zeros((max_rating - min_rating + 1, max_rating - min_rating + 1))
        for i in range(weights.shape[0]):
            for j in range(weights.shape[1]):
                weights[i, j] = (i - j) ** 2
        metrics['quadratic_kappa'] = float(cohen_kappa_score(y_true, y_pred, weights='quadratic'))
    except:
        metrics['quadratic_kappa'] = 0.0

    return metrics

In [52]:
results = evaluate_rag_model_batch(test_df, rag_chain, max_samples=500, batch_size=25, base_sleep_sec=10)

Парсинг ответов: 100%|██████████| 500/500 [00:00<00:00, 274030.05it/s]


In [53]:
results

{'mae': 0.734,
 'mse': 1.434,
 'rmse': 1.1974973903938162,
 'accuracy_exact': 0.516,
 'accuracy_tol1': 0.816,
 'accuracy_tol2': 0.952,
 'pearson_corr': 0.9227292575459591,
 'spearman_corr': 0.9179564342183849,
 'cohen_kappa': 0.4633527590764346,
 'f1_macro': 0.5166843132413161,
 'f1_micro': 0.516,
 'f1_weighted': 0.5105127617077038,
 'precision_macro': 0.5606411226933992,
 'recall_macro': 0.5385725214016519,
 'quadratic_kappa': 0.9119393966190623}

## Без RAG

Сравним с вариантом без RAG.

In [54]:
no_rag_prompt = ChatPromptTemplate.from_template("""
You are a movie review rating assistant.

You are given:
- A NEW REVIEW written by a user.

Use the CONTEXT and the rating scale below to infer an appropriate rating from 1 to 10
for the NEW REVIEW and explain your reasoning.

Rating scale for movie reviews (1–10):

1 — Awful: almost no redeeming qualities, extremely negative experience.
2 — Very bad: many serious problems, very few minor positives.
3 — Bad: mostly negative, some aspects may be tolerable but overall disappointing.
4 — Below average: noticeable flaws, more negatives than positives, would not recommend.
5 — Mixed/average: clear pros and cons, watchable but not impressive.
6 — Decent: slightly above average, more positives than negatives, some enjoyable elements.
7 — Good: solid overall impression, flaws are minor and do not ruin the experience.
8 — Very good: strong positive emotions, well-made, would recommend to most people.
9 — Excellent: highly engaging, memorable, only very minor issues.
10 — Outstanding/masterpiece: exceptional in almost every aspect, would strongly recommend and rewatch.

NEW REVIEW:
{question}

Return:
1) "Rating: X" where X is an integer from 1 to 10.
2) "Explanation: ..." — short explanation based on similarities with CONTEXT and the rating scale.

Answer strictly in English.
""".strip())

no_rag_chain = (
    no_rag_prompt
    | light_llm
)

 Посмотрим на пример генерации.

In [55]:
new_review = "The movie was visually beautiful, but the story was slow and I got bored by the end."
resp = no_rag_chain.invoke(new_review)
print(resp.content)

Rating: 6
Explanation: The movie was visually beautiful, which is a positive aspect. However, the slow story and the fact that the user got bored by the end indicate that there were more negatives than positives in the overall experience. This aligns with the "Decent" category on the rating scale, which is slightly above average, with more positives than negatives but not impressive.


Вычислим результаты.

In [56]:
results_no_rag = evaluate_rag_model_batch(test_df, no_rag_chain, max_samples=500, batch_size=25, base_sleep_sec=10)

Парсинг ответов: 100%|██████████| 500/500 [00:00<00:00, 331775.35it/s]


In [57]:
results_no_rag

{'mae': 0.936,
 'mse': 1.844,
 'rmse': 1.3579396157414365,
 'accuracy_exact': 0.362,
 'accuracy_tol1': 0.768,
 'accuracy_tol2': 0.962,
 'pearson_corr': 0.899846836308257,
 'spearman_corr': 0.8944400789288478,
 'cohen_kappa': 0.2925480246787636,
 'f1_macro': 0.3480404985383173,
 'f1_micro': 0.362,
 'f1_weighted': 0.32987306457691207,
 'precision_macro': 0.41575046524597037,
 'recall_macro': 0.3864181196693286,
 'quadratic_kappa': 0.8847811091005429}

## Сравнительная таблица

Теперь сравним метрики c и без RAG.

In [58]:
comparison_data = []
for metric in results.keys():
    rag_value = results[metric]
    no_rag_value = results_no_rag[metric]

    if metric in ['mae', 'mse', 'rmse']:
        improvement = ((no_rag_value - rag_value) / no_rag_value) * 100
        improvement_type = "снижение"
    else:
        improvement = ((rag_value - no_rag_value) / no_rag_value) * 100
        improvement_type = "улучшение"

    if 'mae' in metric or 'mse' in metric or 'rmse' in metric:
        category = "📈 Метрики ошибок"
    elif 'accuracy' in metric or 'kappa' in metric:
        category = "🎯 Метрики точности"
    elif 'f1' in metric or 'precision' in metric or 'recall' in metric:
        category = "🎭 Метрики классификации"
    elif 'corr' in metric:
        category = "🔗 Метрики корреляции"
    else:
        category = "📊 Прочие метрики"

    comparison_data.append({
        'Категория': category,
        'Метрика': metric,
        'С RAG': rag_value,
        'Без RAG': no_rag_value,
        'Улучшение (%)': improvement,
        'Тип улучшения': improvement_type,
        'Абсолютное изменение': rag_value - no_rag_value
    })

df_comparison = pd.DataFrame(comparison_data)

category_order = ["📈 Метрики ошибок", "🎯 Метрики точности", "🎭 Метрики классификации", "🔗 Метрики корреляции", "📊 Прочие метрики"]
df_comparison['Категория'] = pd.Categorical(df_comparison['Категория'], categories=category_order, ordered=True)
df_comparison = df_comparison.sort_values(['Категория', 'Улучшение (%)'], ascending=[True, False])

def format_value(val, metric):
    if 'mae' in metric or 'mse' in metric or 'rmse' in metric:
        return f"{val:.3f}"
    elif 'kappa' in metric or 'corr' in metric or 'f1' in metric or 'precision' in metric or 'recall' in metric:
        return f"{val:.3f}"
    elif 'accuracy' in metric:
        return f"{val:.3f}"
    else:
        return f"{val:.3f}"

df_comparison['С RAG'] = [format_value(row['С RAG'], row['Метрика']) for _, row in df_comparison.iterrows()]
df_comparison['Без RAG'] = [format_value(row['Без RAG'], row['Метрика']) for _, row in df_comparison.iterrows()]
df_comparison['Улучшение (%)'] = [f"+{row['Улучшение (%)']:.1f}%" if row['Улучшение (%)'] > 0 else f"{row['Улучшение (%)']:.1f}%" for _, row in df_comparison.iterrows()]

print("="*80)
print("📊 СРАВНИТЕЛЬНАЯ ТАБЛИЦА: RAG vs БЕЗ RAG")
print("="*80)

current_category = ""
for _, row in df_comparison.iterrows():
    if row['Категория'] != current_category:
        print(f"\n{row['Категория']}")
        print("-" * 80)
        current_category = row['Категория']

    improvement = float(row['Улучшение (%)'].replace('+', '').replace('%', ''))
    if improvement > 50:
        emoji = "🚀"
    elif improvement > 25:
        emoji = "⭐"
    elif improvement > 10:
        emoji = "✅"
    else:
        emoji = "➡️"

    print(f"{emoji} {row['Метрика']:25} | {row['С RAG']:8} | {row['Без RAG']:8} | {row['Улучшение (%)']:10}")

print("\n" + "="*80)

📊 СРАВНИТЕЛЬНАЯ ТАБЛИЦА: RAG vs БЕЗ RAG

📈 Метрики ошибок
--------------------------------------------------------------------------------
✅ mse                       | 1.434    | 1.844    | +22.2%    
✅ mae                       | 0.734    | 0.936    | +21.6%    
✅ rmse                      | 1.197    | 1.358    | +11.8%    

🎯 Метрики точности
--------------------------------------------------------------------------------
🚀 cohen_kappa               | 0.463    | 0.293    | +58.4%    
⭐ accuracy_exact            | 0.516    | 0.362    | +42.5%    
➡️ accuracy_tol1             | 0.816    | 0.768    | +6.2%     
➡️ quadratic_kappa           | 0.912    | 0.885    | +3.1%     
➡️ accuracy_tol2             | 0.952    | 0.962    | -1.0%     

🎭 Метрики классификации
--------------------------------------------------------------------------------
🚀 f1_weighted               | 0.511    | 0.330    | +54.8%    
⭐ f1_macro                  | 0.517    | 0.348    | +48.5%    
⭐ f1_micro           

Видно значительное улучшение по всем метрикам при использовании RAG. По MAE ошибка составляет меньше 0.5 балла, при этом оценки фильма неразличны при этом условии, что все они целые. MSE и RMSE дают 1 балл, что является допустимым отклонением. Метрики точности также дают прекрасное качество, особенно учитывая неидеальность тестовых данных.

Также мы применили метрики по задаче многоклассовой классификации, потому что наш анализатор как раз классифицирует отзывы по группам с той или иной оценкой. Тут виден колоссальный прирост качества с RAG по всем метрикам.

## Выгрузка базы данных

In [ ]:
vectorstore.persist()

/tmp/ipython-input-398866168.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
!zip -r chroma_db.zip ./chroma_db

  adding: chroma_db/ (stored 0%)
  adding: chroma_db/57f5bfa3-f873-42d3-acb7-0cd799e1e60b/ (stored 0%)
  adding: chroma_db/57f5bfa3-f873-42d3-acb7-0cd799e1e60b/index_metadata.pickle (deflated 45%)
  adding: chroma_db/57f5bfa3-f873-42d3-acb7-0cd799e1e60b/link_lists.bin (deflated 78%)
  adding: chroma_db/57f5bfa3-f873-42d3-acb7-0cd799e1e60b/data_level0.bin (deflated 17%)
  adding: chroma_db/57f5bfa3-f873-42d3-acb7-0cd799e1e60b/length.bin (deflated 35%)
  adding: chroma_db/57f5bfa3-f873-42d3-acb7-0cd799e1e60b/header.bin (deflated 58%)
  adding: chroma_db/chroma.sqlite3 (deflated 47%)


In [ ]:
files.download("chroma_db.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>